## Deep Learning Basics with PyTorch
### Part III – Supervised Deep Learning in Practice
## Chapter 11 — Deeper Architectures

In [1]:
# !pip -q install torch numpy matplotlib  # install dependencies if needed
import matplotlib.pyplot as plt  # plotting utilities
import numpy as np  # numerical helpers
import torch  # tensor library
import torch.nn.functional as F  # functional conv helpers
from IPython import get_ipython  # notebook runtime access
plt.style.use("seaborn-v0_8")  # consistent styling
get_ipython().run_line_magic("config", "InlineBackend.figure_format = 'retina'")  # crisp notebook figures

## Convolution (toy example)

In [2]:
x = torch.zeros(1, 1, 6, 6)  # blank canvas
x[:, :, 2:4, 2:4] = 1.0  # add bright 2x2 square in the centre
kernel = torch.tensor([[0.0, 1.0, 0.0], [1.0, -4.0, 1.0], [0.0, 1.0, 0.0]]).view(1, 1, 3, 3)  # laplacian-style filter
y = F.conv2d(x, kernel, stride=1, padding=0)  # valid convolution
print("input shape:", tuple(x.shape))  # confirm input shape
print("output shape:", tuple(y.shape))  # confirm output shape
print("output map:\n", y.squeeze())  # display response values

input shape: (1, 1, 6, 6)
output shape: (1, 1, 4, 4)
output map:
 tensor([[ 0.,  1.,  1.,  0.],
        [ 1., -2., -2.,  1.],
        [ 1., -2., -2.,  1.],
        [ 0.,  1.,  1.,  0.]])


## Padding vs stride (shapes)

In [3]:
sample = torch.randn(1, 1, 5, 5)  # random 5x5 input
kernel = torch.ones(1, 1, 3, 3)  # 3x3 averaging kernel
no_pad = F.conv2d(sample, kernel, stride=1, padding=0)  # valid conv
same_pad = F.conv2d(sample, kernel, stride=1, padding=1)  # keep size
stride_two = F.conv2d(sample, kernel, stride=2, padding=0)  # stride-2 conv
print("no padding:", tuple(no_pad.shape[-2:]))  # show spatial dims
print("padding=1:", tuple(same_pad.shape[-2:]))  # show same-size dims
print("stride=2:", tuple(stride_two.shape[-2:]))  # show downsampled dims

no padding: (3, 3)
padding=1: (5, 5)
stride=2: (2, 2)


## Tiny CNN (shape tracing)

In [4]:
from torch import nn  # neural network modules
class TinyCNN(nn.Module):  # simple two-block CNN
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(  # feature extractor
            nn.Conv2d(1, 8, 3, padding=1),  # conv block 1 keeps resolution
            nn.ReLU(),  # nonlinearity
            nn.MaxPool2d(2),  # downsample by 2
            nn.Conv2d(8, 16, 3, padding=1),  # conv block 2
            nn.ReLU(),  # nonlinearity
            nn.MaxPool2d(2),  # downsample by 2 again
        )
        self.classifier = nn.Sequential(  # linear head
            nn.Flatten(),  # flatten to (B, features)
            nn.Linear(16 * 7 * 7, 10),  # map to logits
        )
    def forward(self, x):  # forward propagation
        x = self.features(x)  # apply conv blocks
        return self.classifier(x)  # project to class scores
model = TinyCNN()  # instantiate network
sample = torch.randn(4, 1, 28, 28)  # mini-batch of images
feats = model.features(sample)  # extract feature maps
logits = model(sample)  # run full network
print("input:", tuple(sample.shape))  # show input shape
print("features:", tuple(feats.shape))  # show feature map shape
print("logits:", tuple(logits.shape))  # show classifier output

input: (4, 1, 28, 28)
features: (4, 16, 7, 7)
logits: (4, 10)
